In [480]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import pandas as pd
import os
import codecs
from sets import Set
import csv
import sys
import commands
import datetime


In [481]:
#------------------------------ DISCENTES---------------------------------------------------------------------------

In [482]:

var_dis = '/var/tmp/solr_front/collections/capes/discentes/download/'
df_auxiliar = []

print 'Lendo os arquivos CAPES Discentes......'
for root, dirs, files in os.walk(var_dis):
    for file in files:
        print file
        arquivo = codecs.open(os.path.join(root, file), 'r')  # , encoding='latin-1')
        input_lenght = int(commands.getstatusoutput('cat ' + os.path.join(root, file) + ' |wc -l ')[1])
        print('Lendo o arquivo {}.......com o total de {} linhas.'.format(file, input_lenght - 1))
        df_auxiliar.append(pd.read_csv(arquivo, sep=';', low_memory=False, encoding='cp1252'))
        #df_auxiliar = pd.read_csv(arquivo, sep=';', nrows=10000, chunksize=1000, encoding='cp1252', low_memory=False)

df_discentes = pd.concat(df_auxiliar, sort=True)



Lendo os arquivos CAPES Discentes......
Discentes_2013_2015.csv
Lendo o arquivo Discentes_2013_2015.csv.......com o total de 956091 linhas.
Discentes_2016_2017.csv
Lendo o arquivo Discentes_2016_2017.csv.......com o total de 732821 linhas.


In [428]:
df_discentes.count()

AN_BASE                           1688912
CD_AREA_AVALIACAO                 1688912
CD_CONCEITO_CURSO                 1688912
CD_CONCEITO_PROGRAMA              1688912
CD_PROGRAMA_IES                   1688912
CS_STATUS_JURIDICO                1688912
DS_DEPENDENCIA_ADMINISTRATIVA     1688912
DS_FAIXA_ETARIA                   1688912
DS_GRAU_ACADEMICO_DISCENTE        1688912
DS_TIPO_NACIONALIDADE_DISCENTE    1688912
DT_MATRICULA_DISCENTE             1688912
DT_SITUACAO_DISCENTE              1688912
ID_ADD_FOTO_PROGRAMA              1688912
ID_ADD_FOTO_PROGRAMA_IES          1688912
ID_PESSOA                         1688912
NM_AREA_AVALIACAO                 1688912
NM_DISCENTE                       1688912
NM_ENTIDADE_ENSINO                1688912
NM_GRANDE_AREA_CONHECIMENTO       1688912
NM_GRAU_PROGRAMA                  1688912
NM_MODALIDADE_PROGRAMA            1688912
NM_MUNICIPIO_PROGRAMA_IES         1688912
NM_ORIENTADOR                     1688912
NM_PAIS_NACIONALIDADE_DISCENTE    

In [429]:
#----------- RENOMEAR NM_ENTIDADE_ENSINO DA FIOCRUZ PARA FUNDACAO OSWALDO CRUZ (FIOCRUZ)  ----------------

In [483]:
df_discentes.replace('FUNDACAO OSWALDO CRUZ', 'FUNDACAO OSWALDO CRUZ (FIOCRUZ)', inplace=True)

In [484]:
# -------- ADICIONANDO OS CAMPOS AO DF_DISCENTES PARA FAZER O MERGE COM CADASTRO DE IES -------------

In [485]:
df_discentes['SG_ENTIDADE_ENSINO_Capes'] = df_discentes['SG_ENTIDADE_ENSINO']
df_discentes['NM_ENTIDADE_ENSINO_Capes'] = df_discentes['NM_ENTIDADE_ENSINO']

In [407]:
# ---------------------------   PROGRAMAS ---------------------------------------------------------------

In [486]:
var_prog = '/var/tmp/solr_front/collections/capes/programas/download/'
df_aux = []
print 'Lendo os arquivos do CAPES programas....'
for root, dirs, files in os.walk(var_prog):

    for file in files:
        print file
        arquivo = codecs.open(os.path.join(root, file), 'r')  # , encoding='latin-1')
        input_lenght = int(commands.getstatusoutput('cat ' + os.path.join(root, file) + ' |wc -l ')[1])
        print 'Arquivo de entrada possui {} linhas de informacao'.format(int(input_lenght) - 1)
        df_aux.append(pd.read_csv(arquivo, sep=';', low_memory=False, encoding='cp1252'))
        #df_auxiliar = pd.read_csv(arquivo, sep=';', nrows=10000, chunksize=1000, encoding='cp1252', low_memory=False)

df_programas = pd.concat(df_aux, sort=True)

Lendo os arquivos do CAPES programas....
2014.csv
Arquivo de entrada possui 3765 linhas de informacao
2013.csv
Arquivo de entrada possui 3568 linhas de informacao
2015.csv
Arquivo de entrada possui 3946 linhas de informacao
2016.csv
Arquivo de entrada possui 4186 linhas de informacao
2017.csv
Arquivo de entrada possui 4345 linhas de informacao


In [487]:
# drop de algumas colunas de programas que geram duplicidade com discentes no momento do merge

In [488]:

df_programas = df_programas.drop(['NM_GRANDE_AREA_CONHECIMENTO','CD_AREA_AVALIACAO',
'NM_AREA_AVALIACAO', 'SG_ENTIDADE_ENSINO', 'NM_ENTIDADE_ENSINO', 'CS_STATUS_JURIDICO',
'DS_DEPENDENCIA_ADMINISTRATIVA','NM_REGIAO', 'NM_MUNICIPIO_PROGRAMA_IES', 'NM_MODALIDADE_PROGRAMA',
'NM_PROGRAMA_IES', 'SG_UF_PROGRAMA', 'NM_GRAU_PROGRAMA', 'CD_CONCEITO_PROGRAMA',
'ID_ADD_FOTO_PROGRAMA_IES', 'ID_ADD_FOTO_PROGRAMA'], axis=1)



In [411]:
df_programas.count()

ANO_INICIO_PROGRAMA             19810
AN_BASE                         19810
AN_INICIO_CURSO                 19810
CD_PROGRAMA_IES                 19810
DS_CLIENTELA_QUADRIENAL_2017    15465
DS_ORGANIZACAO_ACADEMICA        19810
DS_SITUACAO_PROGRAMA            19810
DT_SITUACAO_PROGRAMA            19810
IN_REDE                         19810
NM_AREA_BASICA                   4345
NM_AREA_CONHECIMENTO            19810
NM_ESPECIALIDADE                19810
NM_PROGRAMA_IDIOMA              19810
NM_SUBAREA_CONHECIMENTO         19810
SG_ENTIDADE_ENSINO_REDE           441
dtype: int64

In [ ]:
df_programas.replace('NÃO SE APLICA', df_programas['NM_AREA_CONHECIMENTO'], inplace=True)

In [ ]:
df_programas


In [489]:
# ----------------- MERGE DE DISCENTES E PROGRAMAS -----------------------------------------------

In [490]:
df_merg_dis_prog = df_discentes.merge(df_programas,on=['AN_BASE', 'CD_PROGRAMA_IES'],suffixes=('_discentes', '_programas'))

In [491]:
#-----------------  cadastro de Instituições--------------------------------------

In [492]:
dir = '/var/tmp/solr_front/collections/capes/instituicoes/download'
for root, dirs, files in os.walk(dir):
    for file in files:
        arquivo = codecs.open(os.path.join(root, file), 'r')  # , encoding='latin-1')
        input_lenght = int(commands.getstatusoutput('cat ' + os.path.join(root, file) + ' |wc -l ')[1])
        print 'Arquivo de entrada possui {} linhas de informacao'.format(int(input_lenght) - 1)
        df_cadastro_ies = pd.read_csv(arquivo, sep=';', low_memory=False, encoding='latin-1')
# eliminando as colunas vazias do csv.
df_cadastro_ies = df_cadastro_ies.dropna(how = 'all', axis = 'columns')
#df_cadastro_ies = df_cadastro_ies.dropna(how = 'all', axis = 'rows')

Arquivo de entrada possui 580 linhas de informacao


In [493]:
# ----- DELETANDO O AN_BASE E DS_DEPENDENCIA_ADMINISTRATIVA PARA NÃO GERAR DUAS COLUNAS IGUAIS APÓS O MERGE ------

In [494]:
df_cadastro_ies = df_cadastro_ies.drop(['AN_BASE', 'DS_DEPENDENCIA_ADMINISTRATIVA', 'CS_STATUS_JURIDICO'], axis=1)

In [495]:
df_final = df_merg_dis_prog.merge(df_cadastro_ies, on=['SG_ENTIDADE_ENSINO_Capes', 'NM_ENTIDADE_ENSINO_Capes'],suffixes=('_disc_prog', '_cad_ies'))


In [452]:
df_final.columns

Index([                       u'AN_BASE',              u'CD_AREA_AVALIACAO',
                    u'CD_CONCEITO_CURSO',           u'CD_CONCEITO_PROGRAMA',
                      u'CD_PROGRAMA_IES',             u'CS_STATUS_JURIDICO',
        u'DS_DEPENDENCIA_ADMINISTRATIVA',                u'DS_FAIXA_ETARIA',
           u'DS_GRAU_ACADEMICO_DISCENTE', u'DS_TIPO_NACIONALIDADE_DISCENTE',
                u'DT_MATRICULA_DISCENTE',           u'DT_SITUACAO_DISCENTE',
                 u'ID_ADD_FOTO_PROGRAMA',       u'ID_ADD_FOTO_PROGRAMA_IES',
                            u'ID_PESSOA',              u'NM_AREA_AVALIACAO',
                          u'NM_DISCENTE',             u'NM_ENTIDADE_ENSINO',
          u'NM_GRANDE_AREA_CONHECIMENTO',               u'NM_GRAU_PROGRAMA',
               u'NM_MODALIDADE_PROGRAMA',      u'NM_MUNICIPIO_PROGRAMA_IES',
                        u'NM_ORIENTADOR', u'NM_PAIS_NACIONALIDADE_DISCENTE',
                      u'NM_PROGRAMA_IES',                      u'NM_REGIAO',

In [531]:
# ----------------  FUNCIONANDO PERFEITAMENTE ------------------------------------------------

In [ ]:
# --------Variável duplicada com o mesmo nome, resolvendo -------------------

In [496]:
df_final['SituacaoDiscente'] = df_final['NM_SITUACAO_DISCENTE']

In [497]:
df_final['SituacaoDiscente'] = df_final['SituacaoDiscente'].apply(lambda x: x.encode('ascii', 'ignore').strip())

In [498]:
df_final['SituacaoDiscente'].unique()

array(['DESLIGADO', 'TITULADO', 'MATRICULADO', 'ABANDONOU',
       'MUDANCA DE NVEL SEM DEFESA', 'MUDANA DE NVEL SEM DEFESA'],
      dtype=object)

In [499]:
df_final['SituacaoDiscente'].replace(['MUDANCA DE NVEL SEM DEFESA','MUDANA DE NVEL SEM DEFESA'], 'MUDANCA DE NIVEL SEM DEFESA', inplace=True)

In [500]:
df_final['SituacaoDiscente'].unique()

array(['DESLIGADO', 'TITULADO', 'MATRICULADO', 'ABANDONOU',
       'MUDANCA DE NIVEL SEM DEFESA'], dtype=object)

In [501]:
#----------------------------------- teste --------------------

In [502]:
total_conceito_programa.value_counts()


4    575150
5    403338
3    344052
6    215855
7    145405
2      4702
1       410
Name: CD_CONCEITO_PROGRAMA, dtype: int64

In [ ]:
#--------------------------------------------------------------

In [ ]:
# -------------  TRABALHANDO COM CAPES EXCLUIDOS - USANDO CD_CONCEITO_PROGRAMA  ---------------------

In [503]:
df_final['CD_CONCEITO_PROGRAMA'].unique()


array([3, 4, 2, 5, 1, 6, 7])

In [354]:
#------------------- teste para criação do campo casos_excluidos de cd conceito programa ------

In [505]:
df_final['Casos_Excluidos_GeoCapes'] = df_final['CD_CONCEITO_PROGRAMA'].apply(lambda x: 'Sim – conceito do programa igual 1 e 2' if x < 3 else 'Não – conceito do programa igual a 3, 4, 5, 7')

In [464]:
teste = df_final['Casos_Excluidos_GeoCapes']

In [465]:
teste.value_counts()

Não – conceito do programa igual a 3, 4, 5, 7    1683800
Sim – conceito do programa igual 1 e 2              5112
Name: Casos_Excluidos_GeoCapes, dtype: int64